In [1]:
import pandas as pd
import numpy as np

In [2]:
low=1
high=15

all_dfs = {}
for i in range(10, 23):
    all_dfs['2014-' + str(i)] = df = pd.DataFrame(np.random.randint(low=low, high=high, size=(20, 2)), columns=['start_station_id', 'end_station_id'])

In [3]:
bike_count = np.empty(shape=(high-low), dtype='int')

for key in list(all_dfs.keys()):
    u_start, c_start = np.unique(all_dfs[key].start_station_id, return_counts=True)
    u_end, c_end = np.unique(all_dfs[key].end_station_id, return_counts=True)
    
    # if a precinct is not the start point for any trip
    if len(set(np.array(range(low, high))) - set(u_start)) > 0:
        for i in list(set(np.array(range(low, high))) - set(u_start)):
            c_start = np.insert(c_start, 0, i)
    
    # if a precinct is not the end point for any trip
    if len(set(np.array(range(low, high))) - set(u_end)) > 0:
        for i in list(set(np.array(range(low, high))) - set(u_end)):
            c_end = np.insert(c_end, 0, i)
    
    bike_count = np.vstack([bike_count, (c_start + c_end)/2])

bike_count = np.einsum('ij->ji', bike_count)

In [4]:
bike_count

array([[4.128860e+06, 6.000000e+00, 1.250000e+01, 1.050000e+01,
        1.050000e+01, 8.000000e+00, 1.200000e+01, 5.500000e+00,
        8.500000e+00, 1.000000e+01, 8.500000e+00, 1.050000e+01,
        1.300000e+01, 5.500000e+00],
       [6.029375e+06, 1.000000e+01, 6.000000e+00, 7.000000e+00,
        5.500000e+00, 7.500000e+00, 6.000000e+00, 8.000000e+00,
        1.100000e+01, 9.000000e+00, 1.100000e+01, 9.000000e+00,
        6.000000e+00, 8.000000e+00],
       [3.801155e+06, 5.000000e+00, 6.500000e+00, 7.000000e+00,
        5.000000e+00, 1.150000e+01, 2.000000e+00, 1.050000e+01,
        5.000000e+00, 8.500000e+00, 4.500000e+00, 3.500000e+00,
        5.000000e+00, 6.500000e+00],
       [5.570652e+06, 1.000000e+00, 1.000000e+00, 1.000000e+00,
        3.000000e+00, 5.500000e+00, 2.500000e+00, 1.000000e+00,
        3.000000e+00, 5.000000e+00, 6.500000e+00, 3.500000e+00,
        4.000000e+00, 9.000000e+00],
       [6.619251e+06, 2.000000e+00, 2.000000e+00, 1.000000e+00,
        1.500000e+00